# Bank Account Fraud Detection

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**Read Data**

In [21]:
# import data
data = pd.read_csv('../data/raw/raw_data.csv')
print(f"data shape: {data.shape}")
data.head()

data shape: (1000000, 32)


,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,...,0,1500.0,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,...,0,200.0,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,...,0,200.0,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,...,0,200.0,0,INTERNET,3.743048,other,0,1,0,0


## Data Cleaning

1. Split into numerical and categorical columns
2. Check number of missing values

In [22]:
# define categorical and numerical columns
categorical_columns = ['fraud_bool', 'payment_type', 'employment_status', 'email_is_free', 'housing_status', 'phone_home_valid', 'phone_mobile_valid', 'has_other_cards', 'foreign_request', 'source', 'device_os', 'keep_alive_session', 'device_distinct_emails_8w']

# define numerical columns, everything else
numerical_columns = [col for col in data.columns if col not in categorical_columns]

In [23]:
# change datatype of categorical columns to 'category'
data[categorical_columns] = data[categorical_columns].astype('category')

In [24]:
# print columnn names
data.columns

Index(['fraud_bool', 'income', 'name_email_similarity',
       'prev_address_months_count', 'current_address_months_count',
       'customer_age', 'days_since_request', 'intended_balcon_amount',
       'payment_type', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
       'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'employment_status',
       'credit_risk_score', 'email_is_free', 'housing_status',
       'phone_home_valid', 'phone_mobile_valid', 'bank_months_count',
       'has_other_cards', 'proposed_credit_limit', 'foreign_request', 'source',
       'session_length_in_minutes', 'device_os', 'keep_alive_session',
       'device_distinct_emails_8w', 'device_fraud_count', 'month'],
      dtype='object')

In [62]:
# columns with potential missing values
na_cols = ['prev_address_months_count', 'current_address_months_count', 'intended_balcon_amount', 'bank_months_count', 'session_length_in_minutes', 'device_distinct_emails_8w']

# check for missing values, if values are -1 or negative, they are considered missing
# if data is negative, it is considered missing
# change devise_distinct_emails_8w to numerical
data['device_distinct_emails_8w'] = data['device_distinct_emails_8w'].astype(int)
missing = data[na_cols]
missing[missing <= 0] = np.nan
missing.isnull().sum()

/var/folders/_3/8z9s_23x6w349w1_9vlqlhzh0000gn/T/ipykernel_69059/1514125876.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing[missing <= 0] = np.nan
/var/folders/_3/8z9s_23x6w349w1_9vlqlhzh0000gn/T/ipykernel_69059/1514125876.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing[missing <= 0] = np.nan


prev_address_months_count       712920
current_address_months_count     13863
intended_balcon_amount          742523
bank_months_count               253635
session_length_in_minutes         2015
device_distinct_emails_8w         6631
dtype: int64

In [63]:
# create dataframe with na_counts, na_percentage
missing_info = missing.isnull().sum().to_frame(name='na_counts')
missing_info['na_proportion'] = round((missing_info['na_counts'] / data.shape[0])*100, 2)
missing_info

,na_counts,na_proportion
prev_address_months_count,712920,71.29
current_address_months_count,13863,1.39
intended_balcon_amount,742523,74.25
bank_months_count,253635,25.36
session_length_in_minutes,2015,0.20
device_distinct_emails_8w,6631,0.66


In [72]:
data['fraud_bool'].value_counts()

fraud_bool
0    988971
1     11029
Name: count, dtype: int64

In [64]:
# concat missing with data['fraud_bool'] along columns
missing_data = pd.concat([missing, data['fraud_bool']], axis=1)
print(f'missing_data shape: {missing_data.shape}')
missing_data.head()

missing_data shape: (1000000, 7)


,prev_address_months_count,current_address_months_count,intended_balcon_amount,bank_months_count,session_length_in_minutes,device_distinct_emails_8w,fraud_bool
0,NaN,25.0,102.453711,9.0,16.224843,1.0,0
1,NaN,89.0,NaN,2.0,3.363854,1.0,0
2,9.0,14.0,NaN,30.0,22.730559,1.0,0
3,11.0,14.0,NaN,1.0,15.215816,1.0,0
4,NaN,29.0,47.152498,26.0,3.743048,1.0,0


In [91]:
data['fraud_bool'].value_counts()

fraud_bool
0    988971
1     11029
Name: count, dtype: int64

In [69]:
# keep rows with missing data
prev_address_months_count_missing = missing_data[missing_data['prev_address_months_count'].isnull()]

prev_address_months_count_missing['fraud_bool'].value_counts() # 10134/11029 = 91.8% fraud cases

fraud_bool
0    702786
1     10134
Name: count, dtype: int64

In [73]:
# keep rows with missing data
current_address_months_count_missing = missing_data[missing_data['current_address_months_count'].isnull()]

current_address_months_count_missing['fraud_bool'].value_counts() # 49/11029 = 0.44% fraud cases

fraud_bool
0    13814
1       49
Name: count, dtype: int64

In [75]:
intended_balcon_amount_missing = missing_data[missing_data['intended_balcon_amount'].isnull()]

intended_balcon_amount_missing['fraud_bool'].value_counts() # 9747/11029 = 88.4% fraud cases

fraud_bool
0    732776
1      9747
Name: count, dtype: int64

In [77]:
bank_months_count_missing = missing_data[missing_data['bank_months_count'].isnull()]

bank_months_count_missing['fraud_bool'].value_counts() # 4140/11029 = 37.5% fraud cases

fraud_bool
0    249495
1      4140
Name: count, dtype: int64

In [80]:
session_length_in_minutes_missing = missing_data[missing_data['session_length_in_minutes'].isnull()]

session_length_in_minutes_missing['fraud_bool'].value_counts() # 18/11029 = 0.16% fraud cases

fraud_bool
0    1997
1      18
Name: count, dtype: int64

In [82]:
device_distinct_emails_8w_missing = missing_data[missing_data['device_distinct_emails_8w'].isnull()]

device_distinct_emails_8w_missing['fraud_bool'].value_counts() # 155/11029 = 1.41% fraud cases

fraud_bool
0    6476
1     155
Name: count, dtype: int64

In [88]:
# find all the rows that have missing values in 'prev_address_months_count' and 'intended_balcon_amount'
combination_missing = missing_data[(missing_data['prev_address_months_count'].isnull()) & (missing_data['intended_balcon_amount'].isnull())]
print(f'combination_missing shape: {combination_missing.shape}')
combination_missing['fraud_bool'].value_counts() # 8943/11029 = 81.1% fraud cases

combination_missing shape: (522549, 7)


fraud_bool
0    513606
1      8943
Name: count, dtype: int64

In [93]:
combination_missing = missing_data[(missing_data['prev_address_months_count'].isnull()) & (missing_data['intended_balcon_amount'].isnull()) & (missing_data['bank_months_count'].isnull())]
print(f'combination_missing shape: {combination_missing.shape}')
combination_missing = combination_missing['fraud_bool'].value_counts()


combination_missing shape: (161867, 7)


In [86]:
missing_data.isna().sum()

prev_address_months_count       712920
current_address_months_count     13863
intended_balcon_amount          742523
bank_months_count               253635
session_length_in_minutes         2015
device_distinct_emails_8w         6631
fraud_bool                           0
dtype: int64

**Conclusion on Missing Values Analysis**

| Columns with Missing Values | % of Fraud Cases that have missing values | % of Non-Fraud Cases that have missing values |
|:----------------------------|:------------------------------------------|:----------------------------------------------|
| prev_address_months_count | 91.8% | 71.1% |
| current_address_months_count | 0.44% | 1.39% |
| intended_balcon_amount | 88.4% | 74.1% |
| bank_months_count | 37.5% | 25.2% |
| session_length_in_minutes | 0.16% | 0.20% |
| device_distinct_emails_8w | 1.41% | 0.66% |

* Insights:
    * Ok to remove `current_address_months_count` and `session_length_in_minutes` because a small proportion of fraud cases are in these missing values.
    * If other columns have missing values, it could be a pattern for fraud.
    * **81.1%** of fraud cases have `prev_address_months_count` and `intended_balcon_amount` as missing values, compared to only **51.9%** of all non-fraud cases as missing values.

## EDA Bank Activity and Device Activity

* Device Activity:
    * `session_length_in_minutes`
    * `device_os`
    * `keep_alive_session`
    * `device_distinct_emails_8w`
    * `device_fraud_count` (can drop this column)
    * `source`
    * `foreign_request`

* Bank Activity:
    * `zip_count_4w`
    * `velocity_6h`
    * `velocity_24h`
    * `velocity_4w`
    * `bank_branch_count_8w`
    * `date_of_birth_distinct_emails_4w`


In [ ]:
# select columns related to device activity
device_activity  = ['fraud_bool', 'session_length_in_minutes', 'device_os', 'keep_alive_session', 'device_distinct_emails_8w', 'source', 'foreign_request']
device_activity_df = data[device_activity]

# print shape
device_activity_df.shape

In [ ]:
# for each column, if it is categorical, plot the value counts, if it is numerical, plot histogram
# put in a 3 x 3 grid, and color code the fraud_bool column
# print all except the first column
plt.figure(figsize=(15, 15))
for i, col in enumerate(device_activity_df.columns[1:]):
    plt.subplot(3, 3, i+1)
    if device_activity_df[col].dtype.name == 'category':
        sns.countplot(data=device_activity_df, x=col, hue='fraud_bool')
    else:
        sns.histplot(data=device_activity_df, x=col, hue='fraud_bool', bins=100)

In [ ]:
# split session_length_in_minutes into 2 groups, fraud and non-fraud
session_length_fraud = device_activity_df[device_activity_df['fraud_bool'] == 1][['session_length_in_minutes', 'fraud_bool']]
sessions_length_non_fraud = device_activity_df[device_activity_df['fraud_bool'] == 0][['session_length_in_minutes', 'fraud_bool']]

# print shapes
print(f"fraud shape: {session_length_fraud.shape}")
print(f"non-fraud shape: {sessions_length_non_fraud.shape}")

In [ ]:
# plot boxplot for non-fraud session length in minutes
# plot side by side, with different colors, on columns 'session_length_in_minutes'
plt.figure(figsize=(6, 3))
sns.boxplot(data=session_length_fraud, x='session_length_in_minutes', color='red')

In [ ]:
# plot boxplot for non-fraud session length in minutes
plt.figure(figsize=(6, 3))
sns.boxplot(data=sessions_length_non_fraud, x='session_length_in_minutes', color='blue')

In [ ]:
# check value_counts for device_distinct_emails_8w
device_activity_df['device_distinct_emails_8w'].value_counts()

In [ ]:
# select device_distinct_emails_8w and fraud_bool columns, and select all the frauds
device_distinct_emails_fraud = device_activity_df[device_activity_df['fraud_bool'] == 1][['device_distinct_emails_8w', 'fraud_bool']]
device_distinct_emails_non_fraud = device_activity_df[device_activity_df['fraud_bool'] == 0][['device_distinct_emails_8w', 'fraud_bool']]

# plot count plot for both fraud and non-fraud, in 1 x 2 grid, provide title for each
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.countplot(data=device_distinct_emails_fraud, x='device_distinct_emails_8w', hue='fraud_bool')
plt.title('Fraud')
plt.subplot(1, 2, 2)
sns.countplot(data=device_distinct_emails_non_fraud, x='device_distinct_emails_8w', hue='fraud_bool')
plt.title('Non-Fraud')


In [ ]:
# use the device_activity_df, combine the device_os and keep_alive_sessions columns
# change device_os and keep_alive_session to str
device_activity_df['device_os'] = device_activity_df['device_os'].astype(str)
device_activity_df['keep_alive_session'] = device_activity_df['keep_alive_session'].astype(str)
device_activity_df['device_os_keep_alive'] = device_activity_df['device_os'] + '_' + device_activity_df['keep_alive_session']

# plot countplot for device_os_keep_alive
plt.figure(figsize=(12, 6))
sns.countplot(data=device_activity_df, x='device_os_keep_alive', hue='fraud_bool')

In [ ]:
# group by device_os_keep_alive
device_os_keep_alive_grouped = device_activity_df.groupby('device_os_keep_alive')
device_os_keep_alive_grouped['fraud_bool'].value_counts()